In [85]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model

import os

In [86]:
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image

In [87]:
TARGET_SIZE = (100, 100)

## Data Augmentation 

In [88]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.2, 1.5]
)

### Generator of train images

In [89]:
data = os.path.join(os.getcwd(), 'dataset/processed_split/train')

train_generator = datagen.flow_from_directory(   
    data, 
    target_size=TARGET_SIZE,
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

num_batches = len(train_generator)
print("Number of batches: ", num_batches)

Found 1280 images belonging to 16 classes.


40

### Generator of test images

In [ ]:
data = os.path.join(os.getcwd(), 'dataset/processed_split/test')

test_generator = datagen.flow_from_directory(
    data, 
    target_size=TARGET_SIZE,
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

num_batches = len(test_generator)
print("Number of batches: ", num_batches)

Son 100 imagenes, de 100x100, en los 3 canales RGB.

## Modelo de Datos

In [98]:
base_filters = 32
w_regularizer = 1e-4

In [99]:
model = tf.keras.Sequential()

# First layer
model.add(Conv2D(filters=base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu', input_shape=(100,100,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# Second layer
model.add(Conv2D(filters=2*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Third layer
model.add(Conv2D(filters=2*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Fourth layer
model.add(Conv2D(filters=2*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

# Fifth layer
model.add(Conv2D(filters=4*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())

# Sixth layer
model.add(Conv2D(filters=4*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# Flatten the output
model.add(Flatten())
model.add(Dense(16, activation='softmax'))

model.summary()


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_24 (Conv2D)              │ (None, 100, 100, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_24 (Activation)      │ (None, 100, 100, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 100, 100, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_25 (Activation)      │ (None, 100, 100, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 50, 50, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 50, 50, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 50, 50, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_26 (Activation)      │ (None, 50, 50, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 50, 50, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 50, 50, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_27 (Activation)      │ (None, 50, 50, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 25, 25, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 25, 25, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 25, 25, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_28 (Activation)      │ (None, 25, 25, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 25, 25, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_29 (Activation)      │ (None, 25, 25, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │       294,928 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 609,040 (2.32 MB)

 Trainable params: 609,040 (2.32 MB)

 Non-trainable params: 0 (0.00 B)

In [100]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [101]:
# NOTE: We are adding 1 to the number of steps per epoch because of an unexpected behavior with the iterator
#      that causes it to skip every other epoch, therefore showing an OUT OF RANGE warning.
#      This is a temporary fix until the issue is resolved.
hist = model.fit(train_generator, steps_per_epoch=len(train_generator)+1, epochs=50, verbose=1)

Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 2/41 ━━━━━━━━━━━━━━━━━━━━ 1:43 3s/step - accuracy: 0.0781 - loss: 85.4018 

KeyboardInterrupt: 

In [ ]:
# Assuming test_images and test_labels are your testing data and labels
test_loss, test_acc = model.evaluate(test_generator, verbose=1)

print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)

2/2 - 1s - 365ms/step - accuracy: 0.7167 - loss: 2.1862

Test accuracy: 0.7166666388511658


## Save model 

In [ ]:
# Save the model
model.save('out/cnn2.keras')